In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import pandas as pd
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.co_visit import CoVisit
from src.models.baseline import popular_items
from src.features.features import feature_engineering
from src.features.utils import build_rank_input
from src.models.tuner import BayesianSearch
from src.models.ranker import Ranker
from src.models.evaluator import Evaluation
from src.models.tracker import launch_mlflow, Logging
from src.models.utils import leave_last_k, set_global_seed

In [2]:
# read config
with open('config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

# ensure reproducibility
set_global_seed(seed=config["general"]["seed"])

# set experiment tracking
launch_mlflow()

# set params
ALGORITHM = "XGBRanker"
neg_sample_k= 15

**Data Preparation & Train/Test Split**

- Load and transform the 3 datasets
- Split whole set into train, validation and test sets by segmenting it temporally

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
dfs = prepare_data(dataframes=dfs, config=config["data_preparation"])

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
df_train, df_valid = leave_last_k(df=df_train, config=config['optimization'])

**Feature Engineering & Optimization**

- Add negative samples, i.e., items likely to be disliked by user
- Feature Engineering - creates cross user-item features for ranking model
- Search which hyper-parameters optimize scoring metric for the given algorithm in the validation set

In [5]:
# negative samples for different sources
neg_sample_1 = popular_items(ui_matrix=df_train, k=neg_sample_k)
neg_sample_2 = CoVisit(methods=["negative"]).fit(ui_matrix=df_train)
neg_sample = pd.concat([neg_sample_1, neg_sample_2], ignore_index=True)

neg_sample = neg_sample[["user_id", "item_id"]]
neg_sample["rating"] = list(config["data_preparation"]["rating_conversion"].keys())[0]

del neg_sample_1, neg_sample_2

In [6]:
# build features for ranking model
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

# add negative samples and merge features
df_train = pd.concat([df_train, neg_sample], ignore_index=True)
df_train, df_valid = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features)
    for df in (df_train, df_valid)
    ]

del neg_sample, user_item_features

In [7]:
# set tuner for hyperparam optimization
tuner = BayesianSearch(
    config["optimization"],
    method="ranker",
    algorithm=ALGORITHM
    )

def objective(trial) -> float:
    return tuner.fit(df_train, df_valid, trial)

# set study
study = optuna.create_study(
    direction=config["optimization"]["ranker"]["direction"],
    sampler=optuna.samplers.TPESampler(seed=config["general"]["seed"])
    )
study.optimize(objective, n_trials= config["optimization"]["n_trials"])

[I 2025-07-25 19:10:43,758] A new study created in memory with name: no-name-8d045930-2e36-4e26-96a8-261f9a0b7d7c
[I 2025-07-25 19:10:49,719] Trial 0 finished with value: 0.9475062672322376 and parameters: {'learning_rate': 0.1878955193048339, 'gamma': 9.50714306409916, 'max_depth': 12, 'subsample': 0.7993292420985183, 'n_estimators': 104}. Best is trial 0 with value: 0.9475062672322376.
[I 2025-07-25 19:11:11,276] Trial 1 finished with value: 0.9495919300106045 and parameters: {'learning_rate': 0.07884126564776513, 'gamma': 0.5808361216819946, 'max_depth': 14, 'subsample': 0.8005575058716043, 'n_estimators': 298}. Best is trial 1 with value: 0.9495919300106045.
[I 2025-07-25 19:11:17,560] Trial 2 finished with value: 0.9478975821845176 and parameters: {'learning_rate': 0.011271662653605422, 'gamma': 9.699098521619943, 'max_depth': 13, 'subsample': 0.6061695553391381, 'n_estimators': 113}. Best is trial 1 with value: 0.9495919300106045.
[I 2025-07-25 19:11:25,002] Trial 3 finished with

**Evaluation & Logging**

- Recover full training set to evaluate results on test set
- Apply negative sampling and perform feature engineering
- Log run's information through mlflow

In [8]:
# get anti test-set, i.e., train & validation sets together
df_train = dfs["data"].merge(
    df_test
    , on=df_test.columns.to_list(), how="left"
    , indicator=True
    )
df_train = df_train[df_train["_merge"] == "left_only"].drop(columns=["_merge"])

In [9]:
# negative samples for different sources
neg_sample_1 = popular_items(ui_matrix=df_train, k=neg_sample_k)
neg_sample_2 = CoVisit(methods=["negative"]).fit(ui_matrix=df_train)
neg_sample = pd.concat([neg_sample_1, neg_sample_2], ignore_index=True)

neg_sample = neg_sample[["user_id", "item_id"]]
neg_sample["rating"] = list(config["data_preparation"]["rating_conversion"].keys())[0]

del neg_sample_1, neg_sample_2

In [10]:
# create features for ranking model
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

# add negative samples and merge features
df_train = pd.concat([df_train, neg_sample], ignore_index=True)
df_train, df_test = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features)
    for df in (df_train, df_test)
    ]

del neg_sample, user_item_features

In [11]:
# set algorithm best hyperparams
hyperparams = (
    config["optimization"]["ranker"][ALGORITHM]["fixed"]
    | study.best_trial.params
)

# fit model on whole training set
clf = Ranker(algorithm=ALGORITHM, params=hyperparams)
clf.fit(X=df_train["X"], y=df_train["y"], group=df_train["group"])
tuner.artifacts["models"][-1] = clf

# test set evaluation
scorer = Evaluation(clf=clf)
tuner.artifacts["metrics_test"] = scorer.fit(train=tuple(df_train.values()), test=tuple(df_test.values()))
tuner.artifacts["metrics_test"]

,ndcg
dataset,
train,0.991994
test,0.951817


In [12]:
# logging experiment
logging = Logging(
    experiment_name="Ranker",
    run_name=ALGORITHM,
    input_sample=df_train["X"].head()
    )
logging.log_run(study=study, tuner=tuner)

🏃 View run 25JUL2025_191241 at: http://127.0.0.1:5000/#/experiments/878870372315907999/runs/87f1f23bcb684e9791faaf34b2e3b249
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/878870372315907999
🏃 View run XGBRanker at: http://127.0.0.1:5000/#/experiments/878870372315907999/runs/3d1abfb0857f453daf475b5a16050721
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/878870372315907999


In [13]:
# shouldn't be done with test set
# recs_score(df_test. iloc[:, :2], df_train.iloc[:, :3])